In [29]:
import pandas as pd 
import numpy as np 
import re

In [3]:
df =pd.read_csv(r"C:\Users\abhis\Downloads\archive (16)\dataset.csv")
df.head(5)

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [4]:
df['review_text'][1]

"This will be more of a ''my experience with this game'' type of review, because saying things like ''great gameplay'' will not suit something I've experienced with Counter-Strike. Here you go:  I remember back in 2002 I was at a friend's house and he was playing a game. I didn't know the name of the game nor I had internet to find it. A few weeks passed by and another friend came over. He didn't have a computer, so he brought a disc with a game in it. He told me that it was one of the best games and from that very moment I knew that it is going to be the game I saw at the other friend's house. When I saw the Counter-Strike logo I was filled with gamegasm (?) and I was so happy. I was playing it hardcore. Made friends, clans, was involved in communities and even made two myself. Counter-Strike is my first game which I played competitively and it was a such an experience. Playing public servers with mods were very fun, but playing it competitively made it very intense and stressful. In 

In [5]:
df['review_score'].value_counts()

review_score
 1    5260420
-1    1156686
Name: count, dtype: int64

In [6]:
df.shape

(6417106, 5)

In [7]:
df.isnull().sum()

app_id               0
app_name        183234
review_text       7305
review_score         0
review_votes         0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6417106 entries, 0 to 6417105
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   app_id        int64 
 1   app_name      object
 2   review_text   object
 3   review_score  int64 
 4   review_votes  int64 
dtypes: int64(3), object(2)
memory usage: 244.8+ MB


In [9]:
df["review_text"] = df["review_text"].astype(str)
df["review_votes"] = df["review_votes"].astype(str)

df.review_text = df.review_text.apply(lambda s : s.strip())
df1 =df[df["review_score"].notnull()]
df1["review_score"] = np.where(df1["review_score"] == -1,0, df1["review_score"])
df1.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [10]:
df1["review_score"].value_counts()

review_score
1    5260420
0    1156686
Name: count, dtype: int64

In [15]:
df1 = df1[df1.review_text != "Early Access Review"]
df1 = df1[~df1.review_text.isin(['nan'])]

df1.drop_duplicates(['review_text', 'review_score'], inplace= True)


In [16]:
df1.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [17]:
df1.shape

(4468409, 5)

In [19]:
import emoji

def extract_emojis(text):
    return "".join(char for char in str(text) if char in emoji.EMOJI_DATA)

df1['emojis_found'] = df1['review_text'].apply(extract_emojis)

# Show only rows that have emojis
df_reviews_with_emojis = df1[df1['emojis_found'] != '']
print(df_reviews_with_emojis[['review_text', 'emojis_found']])

                                               review_text  \
28       Update to my review with some shameless advert...   
33       First of all its my first online game ever. I ...   
41       10/10 Would get 'Fire in the hole' tattooed on...   
52       -No lag -No russian guy saying ♥♥♥♥ ♥♥♥♥♥ -Cla...   
75       This game is awesome.  + still a lot of full s...   
...                                                    ...   
6417042  WORST GAME ♥♥♥♥ING EVER ANYONE WHO PLAYS THIS ...   
6417057  This is the ♥♥♥♥ing worst piece of ♥♥♥♥ game e...   
6417065  If you have a hankerin for a PC game that abso...   
6417066     ♥♥♥♥ this gay ♥♥♥♥ piece of ♥♥♥♥ nigger ♥♥♥♥♥♥   
6417071  gaming at its finest............. if you like ...   

                           emojis_found  
28                                  ♥♥♥  
33                               ♥♥♥♥♥♥  
41                                  ♥♥♥  
52                            ♥♥♥♥♥♥♥♥♥  
75                                 ♥♥♥♥  
...

In [30]:
def replace_hearts_with_PAD(text):
    return re.sub(r"[♥]+", ' **** ' ,text)

In [31]:
import re
def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)

In [25]:
def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

In [26]:
def remove_num(texts):
    output = re.sub(r'\d+', '', texts )
    return output

In [27]:
def unify_whitespaces(text):
    cleaned_string = re.sub(' +', ' ', text )
    return cleaned_string

In [28]:
def remove_punctuation(text):
    result = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"',',') )
    return result

In [21]:
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
nltk.download('stopwords')
from nltk.stem import PorterStemmer

stop = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop ]
    return ' '.join(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def Stemming(text):
    stem = []
    stopword = stopwords.words('english')
    snowball_stemmer = SnowballStemmer('english')
    word_tokens = nltk.word_tokenize(text)
    stemmed_word = [ snowball_stemmer.stem(word) for word in word_tokens ]
    stem = ' '.join(stemmed_word)
    return stem

In [ ]:
def cleaning(df, review):
    df_processed = df.copy()
    df_processed[review] = df_processed['review_text']
    df_processed[review] = df_processed[review].apply(clean)
    df_processed[review] = df_processed[review].apply(deEmojify)
    df_processed[review] = df_processed[review].apply(replace_hearts_with_PAD)
    df_processed[review] = df_processed[review].apply(remove_num)
    df_processed[review] = df_processed[review].apply(remove_punctuation)
    df_processed[review] = df_processed[review].apply(remove_stopwords)
    df_processed[review] = df_processed[review].apply(unify_whitespaces)
    df_processed[review] = df_processed[review].apply(Stemming)
    return df_processed

df_processed = cleaning(df1, 'review_text_clean' )